# Introduccion a SDDP

## Descargar paquete

In [1]:
import Pkg
Pkg.add("SDDP")
Pkg.add("HiGHS")

    Updating registry at `C:\Users\isaia\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\isaia\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\isaia\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\isaia\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\isaia\.julia\environments\v1.10\Manifest.toml`


## Cargar SDDP

In [2]:
using SDDP

## Cargar librerias adicionales

In [3]:
using JuMP, HiGHS, CSV, DataFrames, Statistics, Plots

## Cargar datos de excel

In [4]:
escenarios_df = CSV.read("C:/Users/isaia/Desktop/2024/OperacionEconomicaSistemasElectricos/Tarea_3/escenarios.csv", DataFrame)
println(escenarios_df)

20×3 DataFrame
 Row │ Escenario  Probabilidad  Afluente 
     │ Int64      Float64       Int64    
─────┼───────────────────────────────────
   1 │         1          0.05         5
   2 │         2          0.05        10
   3 │         3          0.05        15
   4 │         4          0.05        20
   5 │         5          0.05        25
   6 │         6          0.05        30
   7 │         7          0.05        35
   8 │         8          0.05        40
   9 │         9          0.05        45
  10 │        10          0.05        50
  11 │        11          0.05        55
  12 │        12          0.05        60
  13 │        13          0.05        65
  14 │        14          0.05        70
  15 │        15          0.05        75
  16 │        16          0.05        80
  17 │        17          0.05        85
  18 │        18          0.05        90
  19 │        19          0.05        95
  20 │        20          0.05       100


## Parametros del Problema

In [5]:
num_weeks = 100
demand = 150.0
thermal_costs = [50.0, 100.0, 150.0]
thermal_capacity = 50.0
hydro_cost = 0.0
max_hydro_storage = 300.0
max_hydro_power = 150.0
initial_storage = 100.0

# Extraer los inflows y las probabilidades
inflows = escenarios_df[!, :Afluente]
probabilities = escenarios_df[!, :Probabilidad]


20-element Vector{Float64}:
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05
 0.05

## Modelo SDDP

In [6]:
# Definir el modelo SDDP
function create_model(iterations)
    model = SDDP.LinearPolicyGraph(
        stages = 100,
        sense = :Min,
        lower_bound = 0, ## Dado que no podemos bajar de costos $0
        optimizer = HiGHS.Optimizer,
    ) do subproblem, t
        ## State variables
        @variable(subproblem, 0 <= volume <= 300, SDDP.State, initial_value = 100)
        ## Control variables
        @variables(subproblem, begin
            thermal_generation1 >= 0
            thermal_generation2 >= 0
            thermal_generation3 >= 0
            hydro_generation >= 0
            hydro_spill >= 0
        end)
        ## Random variables
        @variable(subproblem, inflow)
        
        ## Parametrizar inflows con los datos de escenarios_df
        SDDP.parameterize(subproblem, inflows, probabilities) do ω
            return JuMP.fix(inflow, ω)
        end

        ## Transition function and constraints
        @constraints(
            subproblem,
            begin
                volume.out == volume.in - hydro_generation - hydro_spill + inflow
                demand_constraint, hydro_generation + thermal_generation1 + thermal_generation2 + thermal_generation3 == 150
                max_capacity1, thermal_generation1 <= thermal_capacity
                max_capacity2, thermal_generation2 <= thermal_capacity
                max_capacity3, thermal_generation3 <= thermal_capacity
                max_hydro_generation, hydro_generation <= max_hydro_power
            end
        )
        
        ## Stage-objective
        cost = 50 * thermal_generation1 + 100 * thermal_generation2 + 150 * thermal_generation3
        @stageobjective(subproblem, cost)
    end
    
    # Entrenar el modelo SDDP
    SDDP.train(model; iteration_limit = iterations)
    
    return model
end

create_model (generic function with 1 method)

# Funcion de simulaciones

In [9]:
# Función para simular y analizar resultados
function simulate_and_analyze(model, num_simulations=2000)
    # Simular el modelo para obtener la cota superior
    simulations = SDDP.simulate(
        model,
        num_simulations,  ## Número de realizaciones 2000 especificados por defecto
        [:volume, :thermal_generation1, :thermal_generation2, :thermal_generation3, :hydro_generation, :hydro_spill],
    )
    
    # Analizar los resultados
    volumes = [sim[stage][:volume].out for sim in simulations for stage in 1:100]
    mean_volume = mean(volumes)
    println("Volumen promedio al final de cada semana: ", mean_volume)
    
    # Cotas superior e inferior
    objectives = [sum(stage[:stage_objective] for stage in sim) for sim in simulations]
    μ, ci = SDDP.confidence_interval(objectives, 0.95)
    lower_bound = SDDP.calculate_bound(model)
    upper_bound = μ + ci
    println("Intervalo de confianza: ", μ, " ± ", ci)
    println("Lower bound: ", lower_bound)
    println("Upper bound: ", upper_bound)
    
    # Extraer la evolución del agua almacenada
    stored_water = [sim[week][:volume].out for sim in simulations, week in 1:100]

    # Extraer Costos Futuros del agua almacenada para la primera etapa
    V = SDDP.ValueFunction(model; node=1)
    cost_to_go, gradient = SDDP.evaluate(V, Dict("volume" => initial_storage) )
    println("Costos futuros del agua almacenada para la primera etapa: ", cost_to_go)

    # Costos Marginales variables del agua almacenada para la primera etapa
    println("Costos marginales (variables) del agua almacenada para la primera etapa: ", gradient)

    return (mean_volume, lower_bound, upper_bound, stored_water, cost_to_go, gradient, simulations)
end


simulate_and_analyze (generic function with 2 methods)

## Evaluacion con diferentes iteraciones 

In [11]:
iteration_limits = [5, 20, 50, 100]
results = Dict()
stored_water_evolutions = Dict()
simulations_dict = Dict()

for N in iteration_limits
    println("Resultados para N = $N")
    model = create_model(N)
    mean_volume, lower_bound, upper_bound, stored_water, cost_to_go, gradient, simulations = simulate_and_analyze(model)
    results[N] = (mean_volume, lower_bound, upper_bound, cost_to_go, gradient)
    stored_water_evolutions[N] = stored_water
    simulations_dict[N] = simulations

    # Graficar la evolución del agua almacenada para cada N
    plot()
    for sim in simulations
        plot!(1:100, [stage[:volume].out for stage in sim], alpha=0.3, label=false)
    end
    xlabel!("Semana")
    ylabel!("Evolución del agua almacenada (MWh)")
    title!("Evolución del agua almacenada con N = $N")
    savefig("evolucion_agua_N_$N.png")
end

# Crear la tabla de resultados
df_results = DataFrame(
    N = iteration_limits,
    Lower_Bound = [results[N][2] for N in iteration_limits],
    Upper_Bound = [results[N][3] for N in iteration_limits],
    Costos_Futuros = [results[N][4] for N in iteration_limits],
    Costos_Marginales = [results[N][5] for N in iteration_limits]
)

println(df_results)

# Guardar la tabla de resultados en un archivo CSV
CSV.write("resultados_SDDP.csv", df_results)

# Mostrar resultados de costos futuros y marginales
for N in iteration_limits
    println("Resultados para N = $N")
    println("Cota inferior: ", results[N][2])
    println("Cota superior: ", results[N][3])
    println("Costos futuros del agua almacenada para la primera etapa: ", results[N][5])
    println("Costos marginales (variables) del agua almacenada para la primera etapa: ", results[N][4])
end

Resultados para N = 5
-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 100
  state variables : 1
  scenarios       : 1.26765e+130
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [9, 9]
  AffExpr in MOI.EqualTo{Float64}         : [2, 2]
  AffExpr in MOI.LessThan{Float64}        : [4, 4]
  VariableRef in MOI.GreaterThan{Float64} : [7, 7]
  VariableRef in MOI.LessThan{Float64}    : [1, 2]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 2e+02]
  bounds range     [3e+02, 3e+02]
  rhs range        [5e+01, 2e+02]
-------------------------------------------------------------------
 iteration    simulation      bound    